# Adder

Adding is quite straightforward, with the necessary circuits provides by qiskit

In [12]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import VBERippleCarryAdder

# we do an example of 5 + 3 in a 4-bit registers

bitLength = 4

firstRegister = QuantumRegister(bitLength, 'first')
secondRegister = QuantumRegister(bitLength, 'second')
carryRegister = QuantumRegister(1, 'carry')
ancillaRegister = QuantumRegister(bitLength, 'ancilla')

firstClassicalRegister = ClassicalRegister(bitLength, 'first_classical')
secondClassicalRegister = ClassicalRegister(bitLength, 'second_classical')
ancillaClassicalRegister = ClassicalRegister(bitLength, 'ancilla_classical')


adder = VBERippleCarryAdder(bitLength, name="Adder")
num_qubits = len(adder.qubits)

print(num_qubits)
# define number loader
def loadNumber(circ: QuantumCircuit, register: QuantumRegister, number: int):
    number_in_binary = '0'*(register.size - len(bin(number)[2:])) + bin(number)[2:]
    
    for i in range(len(number_in_binary)):
        if number_in_binary[::-1][i] == '1':
            circ.x(register[i])

firstNumber = 5
secondNumber = 3
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, secondClassicalRegister)

# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)

circ.barrier()

circ.append(adder, list(range(num_qubits)))
circ.barrier()
# circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister[:], secondClassicalRegister)
# circ.measure(ancillaRegister, ancillaClassicalRegister)
circ.draw()


13


░ ┌─────────┐ ░             
             carry: ──────░─┤0        ├─░─────────────
                    ┌───┐ ░ │         │ ░             
           first_0: ┤ X ├─░─┤1        ├─░─────────────
                    └───┘ ░ │         │ ░             
           first_1: ──────░─┤2        ├─░─────────────
                    ┌───┐ ░ │         │ ░             
           first_2: ┤ X ├─░─┤3        ├─░─────────────
                    └───┘ ░ │         │ ░             
           first_3: ──────░─┤4        ├─░─────────────
                    ┌───┐ ░ │         │ ░ ┌─┐         
          second_0: ┤ X ├─░─┤5        ├─░─┤M├─────────
                    ├───┤ ░ │         │ ░ └╥┘┌─┐      
          second_1: ┤ X ├─░─┤6  Adder ├─░──╫─┤M├──────
                    └───┘ ░ │         │ ░  ║ └╥┘┌─┐   
          second_2: ──────░─┤7        ├─░──╫──╫─┤M├───
                          ░ │         │ ░  ║  ║ └╥┘┌─┐
          second_3: ──────░─┤8        ├─░──╫──╫──╫─┤M├
                          ░ │         │ ░  ║  ║  ║ └╥┘
         ancilla_0: ──────░─┤9        ├─░──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║ 
         ancilla_1: ──────░─┤10       ├─░──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║ 
         ancilla_2: ──────░─┤11       ├─░──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║ 
         ancilla_3: ──────░─┤12       ├─░──╫──╫──╫──╫─
                          ░ └─────────┘ ░  ║  ║  ║  ║ 
second_classical: 4/═══════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [13]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'1000': 1000}


In [14]:
def resultToNumber(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        output = splitted_measurements[1]
        results_dict[output] = value
    return results_dict

for firstNumber in range(6):
    for secondNumber in range(6):
        circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.append(adder, list(range(num_qubits)))
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)
        circ.measure(ancillaRegister, ancillaClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = resultToNumber(counts)
        print("{} + {} ".format(firstNumber, secondNumber), processed_counts)
        


0 + 0  {'0000': 1000}
0 + 1  {'0001': 1000}
0 + 2  {'0010': 1000}
0 + 3  {'0011': 1000}
0 + 4  {'0100': 1000}
0 + 5  {'0101': 1000}
1 + 0  {'0001': 1000}
1 + 1  {'0010': 1000}
1 + 2  {'0011': 1000}
1 + 3  {'0100': 1000}
1 + 4  {'0101': 1000}
1 + 5  {'0110': 1000}
2 + 0  {'0010': 1000}
2 + 1  {'0011': 1000}
2 + 2  {'0100': 1000}
2 + 3  {'0101': 1000}
2 + 4  {'0110': 1000}
2 + 5  {'0111': 1000}
3 + 0  {'0011': 1000}
3 + 1  {'0100': 1000}
3 + 2  {'0101': 1000}
3 + 3  {'0110': 1000}
3 + 4  {'0111': 1000}
3 + 5  {'1000': 1000}
4 + 0  {'0100': 1000}
4 + 1  {'0101': 1000}
4 + 2  {'0110': 1000}
4 + 3  {'0111': 1000}
4 + 4  {'1000': 1000}
4 + 5  {'1001': 1000}
5 + 0  {'0101': 1000}
5 + 1  {'0110': 1000}
5 + 2  {'0111': 1000}
5 + 3  {'1000': 1000}
5 + 4  {'1001': 1000}
5 + 5  {'1010': 1000}


In [22]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import VBERippleCarryAdder

# we do an example of 5 + 3 in a 4-bit registers

bitLength = 4

firstRegister = QuantumRegister(bitLength, 'first')
secondRegister = QuantumRegister(bitLength, 'second')
carryRegister = QuantumRegister(1, 'carry')
ancillaRegister = QuantumRegister(bitLength, 'ancilla')

firstClassicalRegister = ClassicalRegister(bitLength, 'first_classical')
secondClassicalRegister = ClassicalRegister(bitLength, 'second_classical')
ancillaClassicalRegister = ClassicalRegister(bitLength, 'ancilla_classical')


adder = VBERippleCarryAdder(bitLength, name="Adder")
num_qubits = len(adder.qubits)

print(num_qubits)
# define number loader
def loadNumber(circ: QuantumCircuit, register: QuantumRegister, number: int):
    number_in_binary = '0'*(register.size - len(bin(number)[2:])) + bin(number)[2:]
    
    for i in range(len(number_in_binary)):
        if number_in_binary[::-1][i] == '1':
            circ.x(register[i])

firstNumber = 6
secondNumber = 3
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister)

# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)

circ.barrier()

circ.append(adder, list(range(num_qubits)))
circ.barrier()
circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister[:], secondClassicalRegister)
# circ.measure(ancillaRegister, ancillaClassicalRegister)
circ.draw()


13


░ ┌─────────┐ ░                         
             carry: ──────░─┤0        ├─░─────────────────────────
                          ░ │         │ ░ ┌─┐                     
           first_0: ──────░─┤1        ├─░─┤M├─────────────────────
                    ┌───┐ ░ │         │ ░ └╥┘┌─┐                  
           first_1: ┤ X ├─░─┤2        ├─░──╫─┤M├──────────────────
                    ├───┤ ░ │         │ ░  ║ └╥┘┌─┐               
           first_2: ┤ X ├─░─┤3        ├─░──╫──╫─┤M├───────────────
                    └───┘ ░ │         │ ░  ║  ║ └╥┘┌─┐            
           first_3: ──────░─┤4        ├─░──╫──╫──╫─┤M├────────────
                    ┌───┐ ░ │         │ ░  ║  ║  ║ └╥┘┌─┐         
          second_0: ┤ X ├─░─┤5        ├─░──╫──╫──╫──╫─┤M├─────────
                    ├───┤ ░ │         │ ░  ║  ║  ║  ║ └╥┘┌─┐      
          second_1: ┤ X ├─░─┤6  Adder ├─░──╫──╫──╫──╫──╫─┤M├──────
                    └───┘ ░ │         │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐   
          second_2: ──────░─┤7        ├─░──╫──╫──╫──╫──╫──╫─┤M├───
                          ░ │         │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
          second_3: ──────░─┤8        ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├
                          ░ │         │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘
         ancilla_0: ──────░─┤9        ├─░──╫──╫──╫──╫──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║ 
         ancilla_1: ──────░─┤10       ├─░──╫──╫──╫──╫──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║ 
         ancilla_2: ──────░─┤11       ├─░──╫──╫──╫──╫──╫──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║ 
         ancilla_3: ──────░─┤12       ├─░──╫──╫──╫──╫──╫──╫──╫──╫─
                          ░ └─────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║ 
 first_classical: 4/═══════════════════════╩══╩══╩══╩══╬══╬══╬══╬═
                                           0  1  2  3  ║  ║  ║  ║ 
second_classical: 4/═══════════════════════════════════╩══╩══╩══╩═
                                                       0  1  2  3

In [23]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'1001 0110': 1000}


In [36]:
def resultToNumber(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        first_number = int(splitted_measurements[1], 2)
        computed_output = int(splitted_measurements[0],2)
        results_dict[(first_number, computed_output)] = value
    return results_dict

for firstNumber in range(2**(bitLength-1)):
    for secondNumber in range(2**(bitLength-1)):
        circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.append(adder, list(range(num_qubits)))
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = resultToNumber(counts)
        print("{} + {} ".format(firstNumber, secondNumber), processed_counts)

0 + 0  {(0, 0): 1000}
0 + 1  {(0, 1): 1000}
0 + 2  {(0, 2): 1000}
0 + 3  {(0, 3): 1000}
0 + 4  {(0, 4): 1000}
0 + 5  {(0, 5): 1000}
0 + 6  {(0, 6): 1000}
0 + 7  {(0, 7): 1000}
1 + 0  {(1, 1): 1000}
1 + 1  {(1, 2): 1000}
1 + 2  {(1, 3): 1000}
1 + 3  {(1, 4): 1000}
1 + 4  {(1, 5): 1000}
1 + 5  {(1, 6): 1000}
1 + 6  {(1, 7): 1000}
1 + 7  {(1, 8): 1000}
2 + 0  {(2, 2): 1000}
2 + 1  {(2, 3): 1000}
2 + 2  {(2, 4): 1000}
2 + 3  {(2, 5): 1000}
2 + 4  {(2, 6): 1000}
2 + 5  {(2, 7): 1000}
2 + 6  {(2, 8): 1000}
2 + 7  {(2, 9): 1000}
3 + 0  {(3, 3): 1000}
3 + 1  {(3, 4): 1000}
3 + 2  {(3, 5): 1000}
3 + 3  {(3, 6): 1000}
3 + 4  {(3, 7): 1000}
3 + 5  {(3, 8): 1000}
3 + 6  {(3, 9): 1000}
3 + 7  {(3, 10): 1000}
4 + 0  {(4, 4): 1000}
4 + 1  {(4, 5): 1000}
4 + 2  {(4, 6): 1000}
4 + 3  {(4, 7): 1000}
4 + 4  {(4, 8): 1000}
4 + 5  {(4, 9): 1000}
4 + 6  {(4, 10): 1000}
4 + 7  {(4, 11): 1000}
5 + 0  {(5, 5): 1000}
5 + 1  {(5, 6): 1000}
5 + 2  {(5, 7): 1000}
5 + 3  {(5, 8): 1000}
5 + 4  {(5, 9): 1000}
5 + 5  

# subtractor

Subtraction can work by 
$$a-b = a + \neg ||b|| +1$$

key thing to take note here is the range of values

In [28]:
firstClassicalRegister = ClassicalRegister(bitLength, 'first_classical')
secondClassicalRegister = ClassicalRegister(bitLength, 'second_classical')
ancillaClassicalRegister = ClassicalRegister(bitLength, 'ancilla_classical')

def resultToNumber(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        output = splitted_measurements[1]
        results_dict[output] = value
    return results_dict

def process_results(counts: dict):
    num = list(counts.keys())[0]
    if num[0] == '0':
        return int(num, 2)
    else:
        return int(num[1:], 2) - 2**len(num[1:])

for firstNumber in range(2**(bitLength-1)): # the first digit is the sign, so does not encode the value
    for secondNumber in range(2**(bitLength-1)):
        circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.x(secondRegister)
        circ.x(carryRegister)
        circ.append(adder, list(range(num_qubits)))
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)
        circ.measure(ancillaRegister, ancillaClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = resultToNumber(counts)
        processed_results = process_results(processed_counts)
        print("{} - {} ".format(firstNumber, secondNumber), processed_counts, processed_results)

0 - 0  {'0000': 1000} 0
0 - 1  {'1111': 1000} -1
0 - 2  {'1110': 1000} -2
0 - 3  {'1101': 1000} -3
0 - 4  {'1100': 1000} -4
0 - 5  {'1011': 1000} -5
0 - 6  {'1010': 1000} -6
0 - 7  {'1001': 1000} -7
1 - 0  {'0001': 1000} 1
1 - 1  {'0000': 1000} 0
1 - 2  {'1111': 1000} -1
1 - 3  {'1110': 1000} -2
1 - 4  {'1101': 1000} -3
1 - 5  {'1100': 1000} -4
1 - 6  {'1011': 1000} -5
1 - 7  {'1010': 1000} -6
2 - 0  {'0010': 1000} 2
2 - 1  {'0001': 1000} 1
2 - 2  {'0000': 1000} 0
2 - 3  {'1111': 1000} -1
2 - 4  {'1110': 1000} -2
2 - 5  {'1101': 1000} -3
2 - 6  {'1100': 1000} -4
2 - 7  {'1011': 1000} -5
3 - 0  {'0011': 1000} 3
3 - 1  {'0010': 1000} 2
3 - 2  {'0001': 1000} 1
3 - 3  {'0000': 1000} 0
3 - 4  {'1111': 1000} -1
3 - 5  {'1110': 1000} -2
3 - 6  {'1101': 1000} -3
3 - 7  {'1100': 1000} -4
4 - 0  {'0100': 1000} 4
4 - 1  {'0011': 1000} 3
4 - 2  {'0010': 1000} 2
4 - 3  {'0001': 1000} 1
4 - 4  {'0000': 1000} 0
4 - 5  {'1111': 1000} -1
4 - 6  {'1110': 1000} -2
4 - 7  {'1101': 1000} -3
5 - 0  {'0101':

In [6]:
firstNumber = 2
secondNumber = 6
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)
circ.barrier()
circ.x(secondRegister)
circ.x(carryRegister)
circ.append(adder, list(range(num_qubits)))
circ.barrier()
circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister, secondClassicalRegister)
circ.measure(ancillaRegister, ancillaClassicalRegister)

job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
processed_counts = resultToNumber(counts)
print("{} - {} ".format(firstNumber, secondNumber), processed_counts)

2 - 6  {'1100': 1000}


# Test Adding constant

In [7]:
firstNumber = 2
secondNumber = 6
constant = 4
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)
circ.barrier()
circ.x(secondRegister)
circ.x(carryRegister)
circ.append(adder, list(range(num_qubits)))
circ.barrier()

circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister, secondClassicalRegister)
circ.measure(ancillaRegister, ancillaClassicalRegister)

circ.draw()


░ ┌───┐┌─────────┐ ░                               »
              carry: ──────░─┤ X ├┤0        ├─░───────────────────────────────»
                           ░ └───┘│         │ ░ ┌─┐                           »
            first_0: ──────░──────┤1        ├─░─┤M├───────────────────────────»
                     ┌───┐ ░      │         │ ░ └╥┘┌─┐                        »
            first_1: ┤ X ├─░──────┤2        ├─░──╫─┤M├────────────────────────»
                     └───┘ ░      │         │ ░  ║ └╥┘┌─┐                     »
            first_2: ──────░──────┤3        ├─░──╫──╫─┤M├─────────────────────»
                           ░      │         │ ░  ║  ║ └╥┘┌─┐                  »
            first_3: ──────░──────┤4        ├─░──╫──╫──╫─┤M├──────────────────»
                           ░ ┌───┐│         │ ░  ║  ║  ║ └╥┘┌─┐               »
           second_0: ──────░─┤ X ├┤5        ├─░──╫──╫──╫──╫─┤M├───────────────»
                     ┌───┐ ░ ├───┤│         │ ░  ║  ║  ║  ║ └╥┘┌─┐            »
           second_1: ┤ X ├─░─┤ X ├┤6  Adder ├─░──╫──╫──╫──╫──╫─┤M├────────────»
                     ├───┤ ░ ├───┤│         │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         »
           second_2: ┤ X ├─░─┤ X ├┤7        ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────»
                     └───┘ ░ ├───┤│         │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      »
           second_3: ──────░─┤ X ├┤8        ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────»
                           ░ └───┘│         │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
          ancilla_0: ──────░──────┤9        ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
          ancilla_1: ──────░──────┤10       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
          ancilla_2: ──────░──────┤11       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
          ancilla_3: ──────░──────┤12       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                           ░      └─────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
  first_classical: 4/════════════════════════════╩══╩══╩══╩══╬══╬══╬══╬══╬══╬═»
                                                 0  1  2  3  ║  ║  ║  ║  ║  ║ »
 second_classical: 4/════════════════════════════════════════╩══╩══╩══╩══╬══╬═»
                                                             0  1  2  3  ║  ║ »
ancilla_classical: 4/════════════════════════════════════════════════════╩══╩═»
                                                                         0  1 »
«                           
«              carry: ──────
«                           
«            first_0: ──────
«                           
«            first_1: ──────
«                           
«            first_2: ──────
«                           
«            first_3: ──────
«                           
«           second_0: ──────
«                           
«           second_1: ──────
«                           
«           second_2: ──────
«                           
«           second_3: ──────
«                           
«          ancilla_0: ──────
«                           
«          ancilla_1: ──────
«                     ┌─┐   
«          ancilla_2: ┤M├───
«                     └╥┘┌─┐
«          ancilla_3: ─╫─┤M├
«                      ║ └╥┘
«  first_classical: 4/═╬══╬═
«                      ║  ║ 
« second_classical: 4/═╬══╬═
«                      ║  ║ 
«ancilla_classical: 4/═╩══╩═
«                      2  3

In [8]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
processed_counts = resultToNumber(counts)
print("{} - {} ".format(firstNumber, secondNumber), processed_counts)

2 - 6  {'1100': 1000}


In [9]:
test_circuit = QuantumCircuit(4,4)
test_circuit.x(0)
test_circuit.measure([0,1], [0,1])
test_circuit.draw()

┌───┐┌─┐
q_0: ┤ X ├┤M├
     └┬─┬┘└╥┘
q_1: ─┤M├──╫─
      └╥┘  ║ 
q_2: ──╫───╫─
       ║   ║ 
q_3: ──╫───╫─
       ║   ║ 
c: 4/══╩═══╩═
       1   0

In [10]:
job = execute(test_circuit, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'0001': 1000}


# QFT subtractor

In [41]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import DraperQFTAdder

num_qubits_for_each_dimension = 4

firstRegister = QuantumRegister(num_qubits_for_each_dimension, 'first')
secondRegister = QuantumRegister(num_qubits_for_each_dimension, 'second')
carryRegister = QuantumRegister(1, 'carry')
firstClassicalRegister = ClassicalRegister(num_qubits_for_each_dimension, 'first_classical')
secondClassicalRegister = ClassicalRegister(num_qubits_for_each_dimension, 'second_classical')

# adder = VBERippleCarryAdder(num_qubits_for_each_dimension, name="Adder")
adder = DraperQFTAdder(num_qubits_for_each_dimension, kind="half",name="Adder")
num_qubits = len(adder.qubits)
print(num_qubits)

circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, firstClassicalRegister, secondClassicalRegister, name="subtractor")
circ.x(secondRegister)
circ.x(carryRegister)
circ.append(adder, list(range(num_qubits)))
circ.draw()

9


┌───┐┌────────┐
             carry: ┤ X ├┤0       ├
                    └───┘│        │
           first_0: ─────┤1       ├
                         │        │
           first_1: ─────┤2       ├
                         │        │
           first_2: ─────┤3       ├
                         │        │
           first_3: ─────┤4 Adder ├
                    ┌───┐│        │
          second_0: ┤ X ├┤5       ├
                    ├───┤│        │
          second_1: ┤ X ├┤6       ├
                    ├───┤│        │
          second_2: ┤ X ├┤7       ├
                    ├───┤│        │
          second_3: ┤ X ├┤8       ├
                    └───┘└────────┘
 first_classical: 4/═══════════════
                                   
second_classical: 4/═══════════════

In [56]:
def process_results(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        first_number = int(splitted_measurements[1], 2)
        two_complement = splitted_measurements[0]
        if two_complement[0] == '0':
            computed_output = int(two_complement[1:], 2)
        else:
            computed_output = int(two_complement[1:], 2) - 2**len(two_complement[1:])
        results_dict[(first_number, computed_output)] = value
    return results_dict

firstRegister = QuantumRegister(num_qubits_for_each_dimension, 'first')
secondRegister = QuantumRegister(num_qubits_for_each_dimension, 'second')
carryRegister = QuantumRegister(1, 'carry')
firstClassicalRegister = ClassicalRegister(num_qubits_for_each_dimension, 'first_classical')
secondClassicalRegister = ClassicalRegister(num_qubits_for_each_dimension, 'second_classical')

adder = DraperQFTAdder(num_qubits_for_each_dimension, kind="half")

for firstNumber in range(2**(num_qubits_for_each_dimension-1)): # the first digit is the sign, so does not encode the value
    for secondNumber in range(2**(num_qubits_for_each_dimension-1)):
        circ = QuantumCircuit(firstRegister, secondRegister, carryRegister, firstClassicalRegister, secondClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.x(secondRegister)
        # circ.x(carryRegister)
        circ.append(adder, firstRegister[:] + secondRegister[:] + [carryRegister[0]])
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = process_results(counts)
        print("{} - {} ".format(firstNumber, secondNumber), counts, processed_counts)

0 - 0  {'1111 0000': 1000} {(0, -1): 1000}
0 - 1  {'1110 0000': 1000} {(0, -2): 1000}
0 - 2  {'1101 0000': 1000} {(0, -3): 1000}
0 - 3  {'1100 0000': 1000} {(0, -4): 1000}
0 - 4  {'1011 0000': 1000} {(0, -5): 1000}
0 - 5  {'1010 0000': 1000} {(0, -6): 1000}
0 - 6  {'1001 0000': 1000} {(0, -7): 1000}
0 - 7  {'1000 0000': 1000} {(0, -8): 1000}
1 - 0  {'0000 0001': 1000} {(1, 0): 1000}
1 - 1  {'1111 0001': 1000} {(1, -1): 1000}
1 - 2  {'1110 0001': 1000} {(1, -2): 1000}
1 - 3  {'1101 0001': 1000} {(1, -3): 1000}
1 - 4  {'1100 0001': 1000} {(1, -4): 1000}
1 - 5  {'1011 0001': 1000} {(1, -5): 1000}
1 - 6  {'1010 0001': 1000} {(1, -6): 1000}
1 - 7  {'1001 0001': 1000} {(1, -7): 1000}
2 - 0  {'0001 0010': 1000} {(2, 1): 1000}
2 - 1  {'0000 0010': 1000} {(2, 0): 1000}
2 - 2  {'1111 0010': 1000} {(2, -1): 1000}
2 - 3  {'1110 0010': 1000} {(2, -2): 1000}
2 - 4  {'1101 0010': 1000} {(2, -3): 1000}
2 - 5  {'1100 0010': 1000} {(2, -4): 1000}
2 - 6  {'1011 0010': 1000} {(2, -5): 1000}
2 - 7  {'1010 